# Метрики эффективности моделей машинного обучения

Метрики качества — численные показатели, которые измеряют, насколько хорошо модель предсказывает. В задачах классификации это, например, accuracy, precision, recall, F1-score, ROC-AUC. Для регрессии — MSE, RMSE, MAE и др.

# Из методички

**Методические указания**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Datasets/heart.csv")
data.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
y = data["output"]
x = data.drop("output", axis=1)

Обучим модель логистической регрессии.

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression().fit(x, y)
logistic.score(x, y)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8547854785478548

Модель показывает более 85% точности. Но оценка оптимистична, так как модель оценена на тех же данных, на которых училась.

Для оценки модели необходимо использовать данные, на которых она не обучалась. Для этого нужно разделить исходный датасет на две части. Обучающая выборка - для нахождения оптимальных значений внутренних параметров модели, тестовая выборка - для оценки качества полученной модели.

**Разделение выборки**

Обучающая и тестовая выборки должны быть непересекающимися. Самый простой способ разделить датасет: какое-то кол-во объектов в обучающую, другую - в тестовую.

In [ ]:
data.shape

(303, 14)

In [ ]:
x_train, y_train = x[:200], y[:200]

Первые 200 строк - обучающая выборка. Тестовая - оставшиеся строки.

In [ ]:
x_train.shape, y_train.shape

((200, 13), (200,))

In [ ]:
x_test, y_test = x[200:], y[200:]
x_test.shape, y_test.shape

((103, 13), (103,))

In [ ]:
logistic_test = LogisticRegression().fit(x_train, y_train)
logistic_test.score(x_train, y_train), logistic_test.score(x_test, y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.9, 0.5436893203883495)

На обучающей выборке точность повысилась до 90%. Эффективность модели на тестовых данных снизилась до 54%. Модель чуть лучше, чем простое угадывание.

Чтобы не задавать вручную кол-во объектов выборок, можно выразить это кол-во через процент от всего объема датасета.

In [ ]:
N = int(x.shape[0] * 0.8)

x_train, y_train, x_test, y_test = x[:N], y[:N], x[N:], y[N:]
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((242, 13), (242,), (61, 13), (61,))

80% объектов для обучающей выборки, 20% объектов для тестовой выборки.

In [ ]:
logistic_test = LogisticRegression().fit(x_train, y_train)
logistic_test.score(x_train, y_train), logistic_test.score(x_test, y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.8884297520661157, 0.6229508196721312)

Обучающая точность - 88%. Тестовая - 62%.

In [ ]:
data.tail()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0
302,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0


Разница между точностями обусловлена тем, что датасет ориентирован по целевой переменной (начало - целевая переменная положительная, конец - отрицательная).

Делить данные на обучающую и тестовую выборки практически всегда нужно случайным образом.

Воспользуемся индексными масками. Заведем массив булевских значений, в которых кол-во истинных значений будет соответствовать желаемому объему обучающей выборки.

In [ ]:
mask = np.array([True] * N + [False] * (y.shape[0] - N))

In [ ]:
from numpy.random import shuffle

shuffle(mask) # перемешаем массив
mask

array([ True,  True,  True, False,  True,  True,  True,  True, False,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True, False,  True, False,  True,  True, False,
        True,  True,  True, False,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True, False, False,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False,  True, False,
        True,  True, False,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,

Можно применить одну и ту же маскук обоим частям массива. Если инвертировать эту маску, то выберем оставшиеся значения.

In [ ]:
x_train = x[mask]
x_train.shape

(242, 13)

In [ ]:
x_train, y_train, x_test, y_test = x[mask], y[mask], x[~mask], y[~mask]
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((242, 13), (242,), (61, 13), (61,))

In [ ]:
logistic_test = LogisticRegression().fit(x_train, y_train)
logistic_test.score(x_train, y_train), logistic_test.score(x_test, y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.8677685950413223, 0.7868852459016393)

Разделение стало более однородным и поэтому точность на тестовых данных повысилась.

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((242, 13), (242,), (61, 13), (61,))

**Построение метрик качества классификации**

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [ ]:
y_test_pred = logistic_test.predict(x_test)
y_train_pred = logistic_test.predict(x_train)

In [ ]:
confusion_matrix(y_train, y_train_pred) # матрица классификации

array([[ 83,  22],
       [ 15, 122]])

In [ ]:
confusion_matrix(y_test, y_test_pred)

array([[28,  5],
       [ 3, 25]])

In [ ]:
print(classification_report(y_test, y_test_pred)) # отчет о классификации

              precision    recall  f1-score   support

           0       0.90      0.85      0.88        33
           1       0.83      0.89      0.86        28

    accuracy                           0.87        61
   macro avg       0.87      0.87      0.87        61
weighted avg       0.87      0.87      0.87        61



In [ ]:
precision_score(y_test, y_test_pred)

0.8333333333333334

In [ ]:
metrics = pd.DataFrame({
    "Train": [
        accuracy_score(y_train, y_train_pred),
        precision_score(y_train, y_train_pred),
        recall_score(y_train, y_train_pred),
        f1_score(y_train, y_train_pred),
    ],
    "Test": [
        accuracy_score(y_test, y_test_pred),
        precision_score(y_test, y_test_pred),
        recall_score(y_test, y_test_pred),
        f1_score(y_test, y_test_pred),
    ],
}, index = ["Accuracy", "Precision", "Recall", "F1"])

metrics

,Train,Test
Accuracy,0.847107,0.868852
Precision,0.847222,0.833333
Recall,0.890511,0.892857
F1,0.868327,0.862069


# Задания для самостоятельного выполнения

*1. Повторите анализ для других видов моделей. Используйте 5-10 разных классов моделей. Подсчитывайте только метрики на тестовой выборке.*

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


models = {
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "Neural Network (MLP)": MLPClassifier(max_iter=500),
    "Support Vector Machine": SVC(probability=True),
    "Linear Discriminant Analysis": LinearDiscriminantAnalysis()
}


metrics = {
    "Model": [],
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1 Score": [],
    "ROC AUC": []
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    try:
        y_prob = model.predict_proba(X_test)[:, 1]
    except:
        y_prob = y_pred

    metrics["Model"].append(name)
    metrics["Accuracy"].append(accuracy_score(y_test, y_pred))
    metrics["Precision"].append(precision_score(y_test, y_pred, zero_division=0))
    metrics["Recall"].append(recall_score(y_test, y_pred, zero_division=0))
    metrics["F1 Score"].append(f1_score(y_test, y_pred, zero_division=0))
    try:
        metrics["ROC AUC"].append(roc_auc_score(y_test, y_prob))
    except:
        metrics["ROC AUC"].append(None)

metrics_df = pd.DataFrame(metrics)
print(metrics_df)


                          Model  Accuracy  Precision   Recall  F1 Score  \
0                 Decision Tree  0.852459   0.925926  0.78125  0.847458   
1                 Random Forest  0.836066   0.843750  0.84375  0.843750   
2           K-Nearest Neighbors  0.688525   0.685714  0.75000  0.716418   
3                   Naive Bayes  0.868852   0.900000  0.84375  0.870968   
4          Neural Network (MLP)  0.786885   0.880000  0.68750  0.771930   
5        Support Vector Machine  0.704918   0.666667  0.87500  0.756757   
6  Linear Discriminant Analysis  0.868852   0.875000  0.87500  0.875000   

    ROC AUC  
0  0.856142  
1  0.928879  
2  0.761315  
3  0.894397  
4  0.918103  
5  0.839440  
6  0.935345  


Accuracy - доля правильно предсказанных наблюдений от общего числа ((TP + TN) / (TP + TN + FP + FN)).

Precision - из всех предсказанных положительных и сколько действительно положительных (TP / (TP + FP)).

Recall - из всех реальных положительных - сколько модель правильно нашла (TP / (TP + FN)).

F1 Score - среднее между Precision и Recall ((Precision * Recall) / (Precision + Recall)).

ROC AUC - способность модели различать классы.

*   LDA

Ищет линейную комбинацию признаков, которая наилучшим образом разделяет классы, при этом предполагает нормальное распределение данных.

LDA показала наилучший баланс между всеми метриками. Самая высокая ROC AUC - модель хорошо различает классы.





*   Random Forest

Каждое дерево обучается на случайной подвыборке данных и признаков, а предсказание - голосованием.

Высокий ROC AUC и хороший F1 Score. Модель показывает хороший результат.




*   Native Bayes

Использует теорему Байеса с предположением независимости признаков.


Простой, но эффективный алгоритм. Немного выше точность, чем у других.



*   Neural Network (MLP)

Состоит из нескольких слоёв нейронов, каждый из которых применяет нелинейные преобразования и обучается через обратное распространение ошибки.

Нейросеть справилась хорошо, высокий ROC AUC - хорошая способность различать классы. Но обучение может занимать больше времени.



*   Decision Tree

Строит дерево решений, выбирая на каждом шаге признак, лучше всего разделяющий данные по какому-либо критерию.

Модель может быть переобучена (слишком хорошо предсказывает обучающие данные, но хуже - тестовые). Высокая точность, но хуже полнота - пропускает часть позитивных классов.



*   Support Vector Machine (SVC)

Ищет гиперплоскость, максимально разделяющую классы с наибольшим зазором.

Невысокая точность, но хорошая полнота - модель отдает предпочтения позитивному классу.



*   K-Nearest Neighbors

Классифицирует объект по большинству классов среди его k ближайших соседей.

метрики показывают, что модель работает хуже остальных. Вероятно, плохо масштабирует пространство признаков (признаки измеряются в разных единицах).



*2. Повторите анализ для другого датасета по вашему выбору. Используйте несколько моделей для сравнения. Используйте датасет для множественной классификации.*

In [ ]:
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler, label_binarize

data = load_wine()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

df.head()


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


Датасет Wine содержит химические характеристики вин трёх разных сортов, произведённых в одном регионе Италии. Всего 178 образцов и 13 числовых признаков. Целевая переменная - класс вина.

In [ ]:
X, y = data.data, data.target

In [ ]:
y_bin = label_binarize(y, classes=[0, 1, 2])

X_train, X_test, y_train, y_test, y_bin_train, y_bin_test = train_test_split(
    X, y, y_bin, test_size=0.3, random_state=42, stratify=y)
# stratify обеспечивает сохранение пропорций классов в выборках

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

models = {
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "Neural Network (MLP)": MLPClassifier(max_iter=1000),
    "Support Vector Machine": SVC(probability=True),
    "Linear Discriminant Analysis": LinearDiscriminantAnalysis()
}

results = []

for name, model in models.items():
    # выбор масштабированных или оригинальных данных
    if name in ["Naive Bayes", "Decision Tree", "Random Forest"]:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)
    else:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_proba = model.predict_proba(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro')
    rec = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    roc = roc_auc_score(y_bin_test, y_proba, average='macro', multi_class='ovr')

    results.append({
        "Model": name,
        "Accuracy": round(acc, 3),
        "Precision": round(prec, 3),
        "Recall": round(rec, 3),
        "F1 Score": round(f1, 3),
        "ROC AUC": round(roc, 3)
    })

df_results = pd.DataFrame(results)
print(df_results)


                          Model  Accuracy  Precision  Recall  F1 Score  \
0                 Decision Tree     0.944      0.951   0.943     0.947   
1                 Random Forest     1.000      1.000   1.000     1.000   
2           K-Nearest Neighbors     0.944      0.944   0.952     0.944   
3                   Naive Bayes     1.000      1.000   1.000     1.000   
4          Neural Network (MLP)     0.944      0.950   0.946     0.947   
5        Support Vector Machine     0.981      0.985   0.978     0.981   
6  Linear Discriminant Analysis     0.981      0.982   0.984     0.983   

   ROC AUC  
0    0.957  
1    1.000  
2    0.995  
3    1.000  
4    0.998  
5    1.000  
6    1.000  


Наивный Байес и Случайный лес показали идеальную точность - отличное распозначание классов.

Нейронная сеть, метод опорных векторов и LDA дали высокие результаты, очень близкие к идеалу.

Дерево решений показало чуть более низкую точность, но справляется все еще хорошо.

Метод ближайших соседей работает хуже остальных.

Все модели показывают отличные результаты, Wine - сбалансированный и хорошо структурированный датасет.

*3. Повторите анализ для датасета, предназначенного для решения задачи регрессии. Используйте все метрики качества регрессии, изученные на лекции. Постройте 5 - 10 разных моделей регрессии.*

MAE - средняя величина абсолютных ошибок между предсказанными и реальными значениями (чем ниже MAE, тем точнее модель).

MSE - средняя величина квадратов ошибок (меньше MSE - точнее модель).

RMSE - корень из MSE, дающий ошибку в тех же единицах, что и целевая переменная (чем ниже - тем точнее модель).

R^2 - коэффициент детерминации - насколько хорошо модель объясняет вариацию в данных (чем ближе к 1, тем лучше модель).

Датасет Boston содержит информацию о различных характеристиках домов в районе Бостона. Он включает 506 образцов и 13 признаков. Цель - предсказать медианную цену домов (MEDV).

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor


data = fetch_openml(name="boston", version=1, as_frame=True)
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "KNN Regressor": KNeighborsRegressor(),
    "SVR": SVR(),
    "MLP Regressor": MLPRegressor(random_state=42, max_iter=2000)
}

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

results = []

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    results.append({
        "Model": name,
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "R²": r2
    })

df_results = pd.DataFrame(results).sort_values(by="RMSE")
print(df_results)


               Model       MAE        MSE      RMSE        R²
2      Random Forest  2.084408   9.621857  3.101912  0.870870
1      Decision Tree  2.540789  11.609079  3.407210  0.844201
5      MLP Regressor  2.313823  12.109569  3.479881  0.837484
3      KNN Regressor  2.628553  18.835039  4.339935  0.747225
0  Linear Regression  3.162710  21.517444  4.638690  0.711226
4                SVR  2.921264  25.957159  5.094817  0.651643


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


Random Forest - самый низкий MAE, а также высокий R^2, что говорит о высоком качестве прогноза.

Decision Tree - хорошие результаты, но хуже случайного леса. Все еще достаточно низкий R^2, что говорит о хорошей способности модели объяснять вариации данных.

MLP - хорошее качество прогноза, однако RMSE выше, модель имеет небольшие ошибки при предсказаниях.

KNN - одна из худших моделей в списке, более высокие MAE, R^2.

Linear Regressor - плохая способность модели объяснять данные, высокое MAE, R^2.

SVR - слабое объяснение данных, низкие MAE, RMSE.

---
Проанализировали данные о болезнях сердца из файла heart.csv. Сначала попытались применить линейную регрессию, но она плохо справилась с задачей классификации. Далее данные разделяли разными способами (поровну, случайно, через train_test_split) и оценивали модели по метрикам accuracy, precision, recall и F1-score.

Построили матрицы ошибок и отчёты о классификации, чтобы понять ошибки моделей. Затем сравнили 9 различных алгоритмов, включая логистическую регрессию, SVM, деревья решений, ансамбли и нейросеть. Лучшие результаты (точность и F1 > 90%) показали ансамбли и нейросеть. Аналогичный анализ провели на датасете wine для многоклассовой классификации.